In [1]:
import numpy as np
from losses import *

2024-12-13 08:57:32.169303: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 08:57:32.177302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734101852.185779  977919 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734101852.188378  977919 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 08:57:32.198202: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
BATCH = 3
SEQ_LEN = 1024
N_CLASSES = 4

# random probabilities
x = np.random.rand(BATCH, SEQ_LEN, N_CLASSES)

# sparsely encoded (random) labels
y = np.random.randint(-1, N_CLASSES, (BATCH, SEQ_LEN))

In [16]:
# Example 1: (batch_size = 1, number of samples = 4)
y_true = [0, 1, 0, 0]
y_pred = [-18.6, 0.51, 2.94, -12.8]
loss = keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True, gamma=2, alpha=0.1, from_logits=True)

In [18]:
y_true = [0, 1, 0, -1]
weights_to_try = [
    [1, 1, 1, 1],
    [1, 1, 1, 0]
]
for wt in weights_to_try:
    print(wt)
    print(loss(y_true, y_pred, sample_weight=np.array(wt)))
    print("=====")


[1, 1, 1, 1]
tf.Tensor(-4.8310595, shape=(), dtype=float32)
=====
[1, 1, 1, 0]
tf.Tensor(-3.6232946, shape=(), dtype=float32)
=====


In [19]:
y_true = [0, 1, 0, 0]
weights_to_try = [
    [1, 1, 1, 1],
    [1, 1, 1, 0]
]
for wt in weights_to_try:
    print(wt)
    print(loss(y_true, y_pred, sample_weight=np.array(wt)))
    print("=====")


[1, 1, 1, 1]
tf.Tensor(0.6088496, shape=(), dtype=float32)
=====
[1, 1, 1, 0]
tf.Tensor(0.4566372, shape=(), dtype=float32)
=====


In [11]:
class MaskedLoss(keras.losses.Loss):
    def __init__(self, loss, mask_value=-1):
        super().__init__()
        self.loss = loss
        self.mask_value = mask_value

    def call(self, y_true, y_pred, sample_weight=None):
        mask = tf.not_equal(y_true, self.mask_value)
        y_true = tf.boolean_mask(y_true, mask)
        y_pred = tf.boolean_mask(y_pred, mask)
        if sample_weight is not None:
            sample_weight = tf.boolean_mask(sample_weight, mask)
        return self.loss(y_true, y_pred, sample_weight)

    def get_config(self):
        return {"loss": self.loss, "mask_value": self.mask_value}

    def from_config(cls, config):
        return cls(**config)

In [24]:
import tensorflow as tf

x = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [25]:
x[1:3] = 45

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [21]:
mbfc = MaskedLoss(keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True, gamma=2, alpha=0.1, from_logits=True))

y_true = [0, 1, 0, 0]
y_pred = [-18.6, 0.51, 2.94, -12.8]
mbfc(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6088495850563049>

In [22]:
mbfc = MaskedLoss(keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True, gamma=2, alpha=0.1, from_logits=True))

y_true = [0, 1, 0, -1]
y_pred = [-18.6, 0.51, 2.94, -12.8]
mbfc(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8117994666099548>

In [23]:
loss(y_true[:-1], y_pred[:-1], )

<tf.Tensor: shape=(), dtype=float32, numpy=0.8117994666099548>

In [14]:
mbfc.loss(y_true, y_pred, sample_weight=np.array([1, 1, 1, 1]))

<tf.Tensor: shape=(), dtype=float32, numpy=8.623503684997559>

In [4]:
# Apply class weight
loss = keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True, gamma=2, from_logits=True)
loss(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5101332664489746>

In [5]:
l = MaskedTemporalCategoricalCrossEntropy(from_logits=False, sparse=True, n_classes=2)


I0000 00:00:1733773396.691563   45434 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9887 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [11]:
y.shape

(3, 1024)

In [13]:
x.shape

(3, 1024, 4)